<a href="https://colab.research.google.com/github/MFahadHussain/MachineLearning/blob/main/Pruning_Only_SimpleCNN_Code_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ✅ Install Required Libraries
!pip install torch torchvision --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.5 MB/s eta 0:00:00


In [ ]:


# ✅ Imports
import torch, torch.nn as nn, torch.nn.functional as F, torchvision
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.nn.utils import prune

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# ✅ CIFAR10 Dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# ✅ SimpleCNN Model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = F.relu(self.fc1(x))
        return self.fc2(x)

model = SimpleCNN().to(device)

# ✅ Train Function
def train(model, epochs=5):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    model.train()
    for epoch in range(epochs):
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# ✅ Evaluate Function
def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"Accuracy: {100 * correct / total:.2f}%")

# ✅ Training Before Pruning
print("Training Before Pruning")
train(model, epochs=5)
print("Evaluation Before Pruning:")
evaluate(model)

# ✅ Apply Pruning (30%)
PRUNE_PERCENTAGE = 0.3
prune.l1_unstructured(model.conv1, name='weight', amount=PRUNE_PERCENTAGE)
prune.l1_unstructured(model.conv2, name='weight', amount=PRUNE_PERCENTAGE)
prune.l1_unstructured(model.fc1, name='weight', amount=PRUNE_PERCENTAGE)
prune.l1_unstructured(model.fc2, name='weight', amount=PRUNE_PERCENTAGE)
print("✅ Pruning Applied (30%)")

# ✅ Fine-Tuning After Pruning
print("Fine-Tuning After Pruning")
train(model, epochs=5)

print("Evaluation After Pruning:")
evaluate(model)


100%|██████████| 170M/170M [00:03<00:00, 43.2MB/s]


Training Before Pruning


In [ ]:
# ✅ Training Before Pruning
print("Training Before Pruning")
train(model, epochs=5)
print("Evaluation Before Pruning:")
evaluate(model)

# ✅ Apply Pruning (30%)
PRUNE_PERCENTAGE = 0.3
prune.l1_unstructured(model.conv1, name='weight', amount=PRUNE_PERCENTAGE)
prune.l1_unstructured(model.conv2, name='weight', amount=PRUNE_PERCENTAGE)
prune.l1_unstructured(model.fc1, name='weight', amount=PRUNE_PERCENTAGE)
prune.l1_unstructured(model.fc2, name='weight', amount=PRUNE_PERCENTAGE)
print("✅ Pruning Applied (30%)")

# ✅ Fine-Tuning After Pruning
print("Fine-Tuning After Pruning")
train(model, epochs=5)

print("Evaluation After Pruning:")
evaluate(model)
